In [1]:
import pandas as pd
import json

In [2]:
data = []
with open('./tmdbKr.json','r',encoding='utf8') as file :
    data= json.load(file)

In [3]:
# 빈 데이터프레임
df = pd.DataFrame(columns=['title','genres','id','release_date','overview','keywords','original_title','vote_average','vote_count','poster_path'])

In [40]:
df

,title,genres,id,release_date,overview,keywords,original_title,vote_average,vote_count,poster_path


In [24]:
#url = "https://image.tmdb.org/t/p/w500/{poster}"

In [16]:
# 영화데이터

# 모든데이터 for문 돌면서
# model == movies.movie 일때
# 각 값들을 뽑아서 새로운 데이터프레임으로 만들고 
# 기존 데이터프레임에 합쳐주는 과정
genre_list = list()
for i in range(len(data)) :
    if data[i]['model'] == 'movies.genre':
        genre_list.append(data[i])
    elif data[i]['model']=='movies.movie':
        title =  data[i]['fields']['title']
        gen = data[i]['fields']['genre_ids']
        #print(gen)
        #break
        gen_id2str = list()
        for gen_ in gen:
            for _a in genre_list:
                if _a['pk'] == gen_:
                    gen_id2str.append(_a['fields']['name'])
                    break    
        
        #print(gen_id2str)
        #break
            
        
        #gen_str = ','.join(str(e) for e in gen) # 여러값이 있어서 해야하는데 이해안감
        id = data[i]['pk']
        if 'release_date' not in data[i]['fields']:
            continue
        release_date = data[i]['fields']['release_date']
        
        overview = data[i]['fields']['overview']
        original_title = data[i]['fields']['original_title']
        vote_average = data[i]['fields']['vote_average']
        vote_count = data[i]['fields']['vote_count']
        
        url = "https://image.tmdb.org/t/p/w500{poster}"
        poster = data[i]['fields']['poster_path']
        poster_path = url,poster
        
        # print(type(poster_path))

        data_json = {'title':[title],'genres':[gen_id2str],'id':[id],'release_date':[release_date],'overview':[overview],'original_title':[original_title],'vote_average':[vote_average],'vote_count':[vote_count],'poster_path':[poster_path]}
        df_tmp= pd.DataFrame(data_json)
        df =  pd.concat([df, df_tmp], ignore_index = True)

In [15]:
df['poster_path'][700]

'url+poster'

In [ ]:
str_pp = list(df['poster_path'])
str_pp = str_pp.apply(lambda x : ('').join(x))

In [21]:
df.isnull().sum()

title             0
genres            0
id                0
release_date      0
overview          0
original_title    0
vote_average      0
vote_count        0
poster_path       0
dtype: int64

In [37]:
#df.to_csv('./tmdbData.csv',encoding='utf8',index=False)
df.to_csv('./test.csv',encoding='utf8',index=False)

## 키워드

In [6]:
# id 가 movie_id 이고 keywords 안 리스트형태로
# keyword_id:value , name:value 형태

# name value만 뽑아서 tmdbData에 컬럼추가

key_data = []
with open('./tmdbKeyword.json','r',encoding='utf8') as file :
    key_data = json.load(file)

In [9]:
key_data[0]

{'id': 640146,
 'keywords': [{'id': 1701, 'name': 'hero'},
  {'id': 3126, 'name': 'ant'},
  {'id': 9663, 'name': 'sequel'},
  {'id': 9715, 'name': 'superhero'},
  {'id': 9717, 'name': 'based on comic'},
  {'id': 18035, 'name': 'family'},
  {'id': 155030, 'name': 'superhero team'},
  {'id': 179430, 'name': 'aftercreditsstinger'},
  {'id': 179431, 'name': 'duringcreditsstinger'},
  {'id': 180547, 'name': 'marvel cinematic universe (mcu)'}]}

In [19]:
# # 키워드 추가

# genre_list = list()
# for i in range(len(data)) :
#     if data[i]['model'] == 'movies.genre':
#         genre_list.append(data[i])
#     elif data[i]['model']=='movies.movie':
#         title =  data[i]['fields']['title']
#         gen = data[i]['fields']['genre_ids']
#         #print(gen)
#         #break
#         gen_id2str = list()
#         for gen_ in gen:
#             for _a in genre_list:
#                 if _a['pk'] == gen_:
#                     gen_id2str.append(_a['fields']['name'])
#                     break 
#         id = data[i]['pk']
#         if 'release_date' not in data[i]['fields']:
#             continue
#         release_date = data[i]['fields']['release_date']
        
#         overview = data[i]['fields']['overview']
#         original_title = data[i]['fields']['original_title']
#         vote_average = data[i]['fields']['vote_average']
#         vote_count = data[i]['fields']['vote_count']
#         poster_path = url,data[i]['fields']['poster_path']
        
        
#         keywords = list()
        
#         if key_data[i]['id'] == data[i]['pk']:
#             keywords.append(key_data['keywords']['name'])
#             #print(keywords)

            
#         data_json = {'title':[title],'genres':[gen_id2str],'id':[id],'release_date':[release_date],'overview':[overview],'keywords':[keywords],'original_title':[original_title],'vote_average':[vote_average],'vote_count':[vote_count],'poster_path':[poster_path]}
#         df_tmp= pd.DataFrame(data_json)
#         df =  pd.concat([df, df_tmp], ignore_index = True)

IndexError: list index out of range

## 영화 검색

In [ ]:
from tmdbv3api import TMDb, Movie

tmdb = TMDb()
tmdb.api_key = "234c446afa7c9156e6c9a101b724f5ae"

movie = Movie()

m = movie.details(111)

print(m.title)
print(m.overview)
print(m.popularity)


Scarface
After getting a green card in exchange for assassinating a Cuban government official, Tony Montana stakes a claim on the drug trade in Miami. Viciously murdering anyone who stands in his way, Tony eventually becomes the biggest drug lord in the state, controlling nearly all the cocaine that comes through Miami. But increased pressure from the police, wars with Colombian drug cartels and his own drug-fueled paranoia serve to fuel the flames of his eventual downfall.
83.822


In [ ]:
# 영화 검색
import requests


def SearchMovies(query, year):  # 영화 제목과 년도 정보를 받음

    params = {"api_key": "234c446afa7c9156e6c9a101b724f5ae",
          "language": "ko",
          "page": "1",
          "query": query,
          "include_adult": "true",
          "year": year}


    url = "https://api.themoviedb.org/3/search/movie" # 정보를 요청할 주소
    resp = requests.get(url, params=params)
    data = resp.json()['results']


    if(len(data) > 0): # 검색해서 나온 데이터가 있을 경우에만

       for item in data : # 반복문 돌면서 영화 이름과 제작년도 출력
           print(item['original_title'], item['release_date']) 

if __name__ == '__main__':
    SearchMovies('존 윅',"2023")

John Wick: Chapter 4 2023-03-22
